### Домашняя работа к уроку 3
### Студент: Абрамов А.В.

In [1]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
#import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [3]:
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [4]:
corpus_title = []
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

99999it [00:03, 28549.53it/s]


In [5]:
corpus_title[7], corpus_text[7]

('Полет российских бомбардировщиков над Карибским морем сняли на\xa0видео',
 'Минобороны опубликовало на YouTube видео полета российских стратегических бомбардировщиков Ту-160, отправленных в Венесуэлу, над водами Карибского моря. Пилоты провели в небе почти 10\xa0часов. «На некоторых этапах маршрута российские ракетоносцы работали совместно с истребителями Су-30 и F-16 Национальной Боливарианской военной авиации. Летчики двух стран отработали взаимодействие при выполнении полетных заданий», — отмечает ведомство под роликом. В южноамериканскую страну были направлены два стратегических ракетоносца Ту-160, тяжелый военный транспортный самолет Ан-124 и дальнемагистральный самолет Ил-62 российских воздушно-космических сил (ВКС). Как передает ТАСС, 14 декабря они покинули государство и полетели на домашнюю базу. Ту-160, сверхзвуковой стратегический бомбардировщик-ракетоносец с изменяемой геометрией крыла, является самым крупным, мощным и тяжелым боевым сверхзвуковым самолетом в мире.')

In [6]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...


In [7]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
preprocess_txt(df['text'].values[0])

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило',
 'сообщить',
 'посол',
 'россия',
 'вена',
 'дмитрий',
 'любинский',
 'итог',
 'встреча',
 'уполномоченный',
 'адвокат',
 'дипмиссия',
 'представитель',
 'прокуратура',
 'страна',
 'передавать',
 'тасс',
 '«действовать',
 'презумпция',
 'невиновность',
 'какихлибо',
 'ограничение',
 'свобода',
 'передвижение',
 'команда',
 'нет»',
 '—',
 'добавить',
 'посольство',
 'международный',
 'союз',
 'биатлонист',
 'ibu',
 'применять',
 'санкция',
 'российский',
 'биатлонист',
 'продолжить',
 'выступление',
 'кубок',
 'мир',
 'полиция',
 'нагрянуть',
 'отель',
 'сборная',
 'россия',
 'хохфильцен',
 'вечером',
 '12',
 'декабрь',
 'написать',
 'биатлонист',
 'александр',
 'логинов',
 'считать',
 'виновный',
 'махинация',
 'переливание',
 'кровь',
 'биатлонист',
 'антон',
 'шипулина',
 'попасть',
 'список',
 'полиция',
 'нанести',
 'отдельны

In [10]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [11]:
sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1
    if counter > 100:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [12]:
sentences[0]

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило',
 'сообщить',
 'посол',
 'россия',
 'вена',
 'дмитрий',
 'любинский',
 'итог',
 'встреча',
 'уполномоченный',
 'адвокат',
 'дипмиссия',
 'представитель',
 'прокуратура',
 'страна',
 'передавать',
 'тасс',
 'действовать',
 'презумпция',
 'невиновность',
 'либо',
 'ограничение',
 'свобода',
 'передвижение',
 'команда',
 'добавить',
 'посольство',
 'международный',
 'союз',
 'биатлонист',
 'применять',
 'санкция',
 'российский',
 'биатлонист',
 'продолжить',
 'выступление',
 'кубок',
 'мир',
 'полиция',
 'нагрянуть',
 'отель',
 'сборная',
 'россия',
 'хохфильцен',
 'вечером',
 '12',
 'декабрь',
 'написать',
 'биатлонист',
 'александр',
 'логинов',
 'считать',
 'виновный',
 'махинация',
 'переливание',
 'кровь',
 'биатлонист',
 'антон',
 'шипулина',
 'попасть',
 'список',
 'полиция',
 'нанести',
 'отдельный',
 'визит',
 'тренироваться

In [13]:
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2)

In [14]:
modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=5, workers=8)